#### SQLite : Tabellen löschen (DROP TABLE IF EXISTS ...)

In [1]:
import time
import sqlite3
from sqlite3 import Error

# database = 'O:/Temporaer/test_SQLite.db'
tabellen = ('tankstellen', 'treibstoffe', 'preise')


def create_connection(db_file):
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(f'\nFehler : Datenbankverbindung gescheitert\nError : "{e}"')
    return None


def drop_table(conn, tabelle):
    sql = f'''DROP TABLE IF EXISTS {tabelle};'''
    # print(sql)
    try:
        cur = conn.cursor()
        cur.execute(sql)
        conn.commit()
        cur.close()
    except Error as e:
        print(f'\nFehler : Tabelle löschen gescheitert\nError : "{e}"')


def _main():
    conn = create_connection(database)
    if conn:
        for tabelle in tabellen:
            drop_table(conn, tabelle)
        conn.close()
    else:
        print(f'Programm abgebrochen\n')


if __name__ == '__main__':
    t0 = time.perf_counter()
    _main()
    print(f'\nLaufzeit : {time.perf_counter() - t0:8,.4f} Sekunden\n')


NameError: name 'database' is not defined

#### SQLite : Prüfung ob Tabellen existieren (SELECT name FROM sqlite_master WHERE type="table" ...)

In [13]:
import time
import sqlite3
from sqlite3 import Error

database = 'O:/Temporaer/test_SQLite.db'
tabellen = ('tankstellen', 'treibstoffe', 'preise', 'new_table1', 'new_table2')


def create_connection(db_file):
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(f'\nFehler : Datenbankverbindung gescheitert\nError : "{e}"')
    return None

def check_tabelle(conn):
    sql = '''SELECT name FROM sqlite_master WHERE type="table"'''    
    cur = conn.cursor()
    cur.execute(sql)
    rows = cur.fetchall()
    cur.close()
    # print(f'Existierende DB-Tabellen : {rows}')
    rows = tuple((n[0] for n in rows))
    # print(f'Existierende DB-Tabellen : {rows}')
    for tabelle in tabellen:
        if tabelle in rows:
            print(f'Tabelle "{tabelle}" existiert bereits in der Datenbank')
        else:
            print(f'Fehler : Tabelle "{tabelle}" nicht gefunden')


def _main():
    conn = create_connection(database)
    if conn:
        check_tabelle(conn)
        conn.close()
    else:
        print(f'Programm abgebrochen\n')


if __name__ == '__main__':
    t0 = time.perf_counter()
    _main()
    print(f'\nLaufzeit : {time.perf_counter() - t0:8,.4f} Sekunden\n')


Tabelle "tankstellen" existiert bereits in der Datenbank
Tabelle "treibstoffe" existiert bereits in der Datenbank
Tabelle "preise" existiert bereits in der Datenbank
Fehler : Tabelle "new_table1" nicht gefunden
Fehler : Tabelle "new_table2" nicht gefunden

Laufzeit :   0.0007 Sekunden



#### SQLite : Tabellen anlegen (CREATE TABLE IF NOT EXISTS ...)

In [2]:
import time
import sqlite3
from sqlite3 import Error

database = 'O:/Temporaer/test_SQLite.db'


def create_connection(db_file):
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(f'\nFehler : Datenbankverbindung gescheitert\nError : "{e}"')
    return None


def create_table(conn, create_table_sql):
    try:
        cur = conn.cursor()
        cur.execute(create_table_sql)
        conn.commit()
        cur.close()
    except Error as e:
        print(f'\nFehler : Neuanlage Tabelle fehlgeschlagen\nError : "{e}"')


def init_database(conn):
    sql_create_tankstellen_table = """
    CREATE TABLE IF NOT EXISTS tankstellen (
        Id      INTEGER PRIMARY KEY ASC AUTOINCREMENT,
        Url     TEXT    NOT NULL
                        UNIQUE ON CONFLICT ROLLBACK,
        Name    TEXT    NOT NULL,
        Plz     TEXT    NOT NULL,
        Stadt   TEXT    NOT NULL,
        Strasse TEXT    NOT NULL
    );
    """
    sql_create_treibstoffe_table = """
    CREATE TABLE IF NOT EXISTS treibstoffe (
        Id          INTEGER PRIMARY KEY ASC AUTOINCREMENT,
        Bezeichnung TEXT    UNIQUE ON CONFLICT ROLLBACK
                            NOT NULL
    );
    """
    sql_create_preise_table = """
    CREATE TABLE IF NOT EXISTS preise (
        Id            INTEGER PRIMARY KEY ASC AUTOINCREMENT,
        Tankstelle_Id INTEGER NOT NULL,
        Treibstoff_Id INTEGER NOT NULL,
        Timestamp     REAL NOT NULL,
        Preis         REAL NOT NULL,
        Datum_Zeit    TEXT NOT NULL,
        FOREIGN KEY (
            Tankstelle_Id
        )
        REFERENCES tankstellen (Id),
        FOREIGN KEY (
            Treibstoff_Id
        )
        REFERENCES treibstoffe (Id) 
    );
    """
    create_table(conn, sql_create_tankstellen_table)
    create_table(conn, sql_create_treibstoffe_table)
    create_table(conn, sql_create_preise_table)
    return


def _main():
    conn = create_connection(database)
    if conn:
        init_database(conn)
        conn.close()
    else:
        print(f'Programm abgebrochen\n')


if __name__ == '__main__':
    t0 = time.perf_counter()
    _main()
    print(f'\nLaufzeit : {time.perf_counter() - t0:8,.4f} Sekunden\n')



Laufzeit :   1.0294 Sekunden



#### SQLite : Alle Datensätze auslesen (SELECT ...)

In [30]:
import time
import sqlite3
from sqlite3 import Error

database = 'O:/Temporaer/test_SQLite.db'


def create_connection(db_file):
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(f'\nFehler : Datenbankverbindung gescheitert\nError : "{e}"')
    return None


def _main():
    sql_1 = '''
    SELECT
    *
    FROM
    View_Super_E5
    ;'''
    sql_2 = '''
    SELECT
    tankstellen.Name, treibstoffe.Bezeichnung,
    preise.Preis, preise.Timestamp,
    preise.Datum_Zeit
    FROM
    tankstellen, treibstoffe, preise
    WHERE
    (treibstoffe.Id = preise.Treibstoff_Id) AND
    (tankstellen.Id = preise.Tankstelle_Id)
    ORDER BY preise.Timestamp DESC
    ;'''
    sql_anweisungen = (sql_1, sql_2)
    conn = create_connection(database)
    if conn:
        for sql in sql_anweisungen:
            cur = conn.cursor()
            cur.execute(sql)
            rows = cur.fetchall()
            cur.close()
            if len(rows):
                for n, row in enumerate(rows):
                    print(row)
                print(f'{n} Datensätze eingelesen\n')
            else:
                print(f'{0} Datensätze eingelesen\n')
        conn.close()
    else:
        print(f'Programm abgebrochen\n')


if __name__ == '__main__':
    t0 = time.perf_counter()
    _main()
    print(f'\nLaufzeit : {time.perf_counter() - t0:8,.4f} Sekunden\n')


('ARAL', 'Super E5', 2.169, 1647613620.0, '18.03.2022 15:27')
('Avanti', 'Super E5', 2.119, 1647613500.0, '18.03.2022 15:25')
('Shell', 'Super E5', 2.149, 1647612960.0, '18.03.2022 15:16')
('ARAL', 'Super E5', 2.179, 1647612480.0, '18.03.2022 15:08')
('ARAL', 'Super E5', 2.139, 1647611040.0, '18.03.2022 14:44')
('ARAL', 'Super E5', 2.149, 1647610620.0, '18.03.2022 14:37')
('Shell', 'Super E5', 2.099, 1647609960.0, '18.03.2022 14:26')
('Avanti', 'Super E5', 2.079, 1647608700.0, '18.03.2022 14:05')
('ARAL', 'Super E5', 2.179, 1647532980.0, '17.03.2022 17:03')
('Shell', 'Super E5', 2.129, 1647532080.0, '17.03.2022 16:48')
('Avanti', 'Super E5', 2.109, 1647530880.0, '17.03.2022 16:28')
('ARAL', 'Super E5', 2.159, 1647526320.0, '17.03.2022 15:12')
11 Datensätze eingelesen

('ARAL', 'Diesel', 2.179, 1647613620.0, '18.03.2022 15:27')
('ARAL', 'Super E10', 2.109, 1647613620.0, '18.03.2022 15:27')
('ARAL', 'Super E5', 2.169, 1647613620.0, '18.03.2022 15:27')
('ARAL', 'ARAL Superplus', 2.239, 16